In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
def cal_length_value(x1, y1, x2, y2):
    # x1, y1 = map(float, input('x1 y1: ').split())
    # x2, y2 = map(float, input('x2 y2: ').split())
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

def cal_area_value(ax, ay, bx, by, cx, cy):
    # ax, ay = map(float, input('ax ay: ').split())
    # bx, by = map(float, input('bx by: ').split())
    # cx, cy = map(float, input('cx cy: ').split())

    a = ((bx - cx)**2 + (by - cy)**2)**0.5
    b = ((ax - cx)**2 + (ay - cy)**2)**0.5
    c = ((ax - bx)**2 + (ay - by)**2)**0.5

    s = (a + b + c) / 2
    S = (s * (s - a) * (s - b) * (s - c))**0.5
    return S

def cal_angle_value(x1, y1, x2, y2, x3, y3):
    a = np.array([x1, y1])
    b = np.array([x2, y2])
    c = np.array([x3, y3])

    vec_a = a - b
    vec_c = c - b

    length_vec_a = np.linalg.norm(vec_a)
    length_vec_c = np.linalg.norm(vec_c)
    inner_product = np.inner(vec_a, vec_c)
    if length_vec_a * length_vec_c == 0:
        print(length_vec_a, length_vec_c)
        degree = 0
    else:
        cos = inner_product / (length_vec_a * length_vec_c)

        rad = np.arccos(cos)

        degree = np.rad2deg(rad)
    return degree

In [36]:
anno = pd.read_csv('annotation_post.csv')
subject_list = anno['name'].unique()

In [37]:
subject_list

array(['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's11',
       's12'], dtype=object)

In [33]:
new_subject_list = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's11',
       's12']

In [38]:
for subject_name in new_subject_list:
    # print('dealing ' + subject_name)
    file_path = f"us_annotation_March/{subject_name}_usraw_2.csv"
    df = pd.read_csv(file_path)
    df = df.loc[df['0.000000000000000000e+00'] != 0]
    df.columns = ['1x', '1y', '2x', '2y', '3x', '3y', '4x', '4y', '5x', '5y']
    df_new = pd.DataFrame(columns=['pitch_name', 'section', 'round', 'length_value_left', 'length_value_right', 'area_value_left', 'area_value_right', 'angle_value_in', 'angle_value_out', 'frame_num'])
    for index, row in df.iterrows():
        idx = index + 1
        length_value_left = cal_length_value(row['2x'], row['2y'], row['5x'], row['5y'])
        length_value_right = cal_length_value(row['3x'], row['3y'], row['5x'], row['5y'])
        area_value_left = cal_area_value(row['1x'], row['1y'], row['2x'], row['2y'], row['5x'], row['5y'])
        area_value_right = cal_area_value(row['3x'], row['3y'], row['4x'], row['4y'], row['5x'], row['5y'])
        angle_value_in = cal_angle_value(row['2x'], row['2y'], row['5x'], row['5y'], row['3x'], row['3y'])
        angle_value_out = cal_angle_value(row['1x'], row['1y'], row['5x'], row['5y'], row['4x'], row['4y'])
        # baseline
        if idx == anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'baseline'), 'start'].values[0]:
            # print(row['1x'])
            df_new.loc[len(df_new.index)] = ['', 'baseline', 0, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
        # before
        elif idx >= anno.loc[(anno['name'] == subject_name) & (anno['cond'] == 'after practice'), 'start'].values[0]:
            df_new.loc[len(df_new.index)] = ['', 'after practice', 4, length_value_left, length_value_right, area_value_left, area_value_right, angle_value_in, angle_value_out, idx]
  # print(len(df))
    df_new.to_csv(subject_name + '_processed_post.csv', index=False)
    print('saved ' + subject_name)

saved s01
saved s02
saved s03
saved s04
saved s05
saved s06
saved s07
saved s08
saved s11
saved s12
